In [1]:
import os, pickle
import numpy as np
import tensorflow as tf
from keras.callbacks import *
#from keras.optimizers.optimizer_experimental.adamw import AdamW

from settings import *
import data
from generator import *
from model import *
from callbacks import *
from utils import *
from tokenizer import *
from tokenizers import NormalizedString
import pretokenizer as mypretk

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:

	try:
		tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
		print('Using GPU :)')

	except RuntimeError as e:
		print(e)

else:
	print('Using CPU :(')

Using GPU :)


In [3]:
dataset = data.parse_dataset(DATASET_PATH)

print('Dataset size:', len(dataset))

print('\n' + dataset[:500])

Importing dataset...
Parsing dataset...
Importing parsed dataset...
Dataset size: 464190689

Ironie : l'article disant qu'on est plus capable de se concentrer plus de 2 pages, en fait 6 !<nl>J'ai pas pu aller plus loin que la page 3, même si c'était intéressant !<eom>Moi-même, j'ai dû me forcer pour arriver jusqu'à la fin. Et pourtant ça ne devient pas moins intéressant. Ca fout les jetons... :)<eom><eod>Service qui sera rendu au contribuable pour la modique some de 70 Millions par an.<eom>Eeeeh oui ! 70 millions pour une loi qui aura pour effet de .<eom><eod>Est-ce qu'elle a vraiment c


In [4]:
words = mypretk.split(dataset[:500])

print(dataset[:500].replace(' ', '_'))
print_tokens(words)

Ironie_:_l'article_disant_qu'on_est_plus_capable_de_se_concentrer_plus_de_2_pages,_en_fait_6_!<nl>J'ai_pas_pu_aller_plus_loin_que_la_page_3,_même_si_c'était_intéressant_!<eom>Moi-même,_j'ai_dû_me_forcer_pour_arriver_jusqu'à_la_fin._Et_pourtant_ça_ne_devient_pas_moins_intéressant._Ca_fout_les_jetons..._:)<eom><eod>Service_qui_sera_rendu_au_contribuable_pour_la_modique_some_de_70_Millions_par_an.<eom>Eeeeh_oui_!_70_millions_pour_une_loi_qui_aura_pour_effet_de_.<eom><eod>Est-ce_qu'elle_a_vraiment_c
Ironie_:_l'article_disant_qu'on_est_plus_capable_de_se_concentrer_plus_de_2_pages,_en_fait_6_!<nl>J'ai_pas_pu_aller_plus_loin_que_la_page_3,_même_si_c'était_intéressant_!<eom>Moi-même,_j'ai_dû_me_forcer_pour_arriver_jusqu'à_la_fin._Et_pourtant_ça_ne_devient_pas_moins_intéressant._Ca_fout_les_jetons..._:)<eom><eod>Service_qui_sera_rendu_au_contribuable_pour_la_modique_some_de_70_Millions_par_an.<eom>Eeeeh_oui_!_70_millions_pour_une_loi_qui_aura_pour_effet_de_.<eom><eod>Est-ce_qu'elle_a_vraiment_

In [5]:
tokenizer = Tokenizer(dataset)
vocab = tokenizer.vocab

print('\nVocab size:', len(vocab), '\n')

pickle.dump(vocab, open(os.path.join(PROCESSED_DATA_DIR, 'vocab.pkl'), 'wb'))

for v in vocab:
	print(f'[{v}]', end = ' ')

Creating vocab...
Pretokenize...
Splitting control characters...
Splitting words...
Splitting multiple spaces...
Nb words: 105721487
Merges...
Max token length: 32
Sorting vocab...
Pretokenize...
Splitting control characters...
Splitting words...
Splitting multiple spaces...
Progress: 100%               

Vocab size: 20000 

['] [.] [,] [ de] [<nl>] [<eom>] [ la] [ le] [ que] [ pas] [ les] [est] [ et] [ à] [ l] [ des] [ un] [ d] [ en] [-] [ c] [ qui] [s] [ pour] [ je] [ qu] [ une] [ a] [ est] [ du] [<eod>] [ ne] [ ça] [ dans] [ ce] [ n] [ plus] [/] [ sur] [ tu] [ mais] [ (] [e] [ ?] [ il] [ "] [ai] [ j] [ au] [>] [ on] [ fait] [ par] [ avec] [ se] [ sont] [ ou] [ si] [ tout] [il] [ bien] [)] [ même] [a] [ y] [ faire] ["] [...] [ comme] [ me] [C] [ s] [ :] [on] [un] [Je] [en] [r] [ quand] [ peut] [ suis] [ aussi] [ peu] [?] [://] [ être] [es] [ ont] [ C] [_] [ leur] [ te] [ !] [ son] [ t] [).] [ Je] [ très] [ gens] [ non] [y] [ ils] [ dire] [é] [nt] [ aux] [J] [ mon] [ m] [ moins] [une]

In [ ]:
if os.path.exists(os.path.join(PROCESSED_DATA_DIR, 'tokens.npy')):
	tokens = np.load(os.path.join(PROCESSED_DATA_DIR, 'tokens.npy'))

else:
	tokens = np.array(tokenizer.encode(dataset).ids)

	if not os.path.exists(PROCESSED_DATA_DIR):
		os.mkdir(PROCESSED_DATA_DIR)

	np.save(os.path.join(PROCESSED_DATA_DIR, "tokens.npy"), tokens)

print('Encoded dataset:')

for token in tokens[:100]:
	print(token, end = " ")

print('...')
print('\n\nDecoded dataset:')
print('"', end = '')

for i in tokens[:100]:
	text = tokenizer.decode([i], False)
	if text == '<eom>':
		print('\n')
	elif text == '<eod>':
		print('\n-------------------\n')
	else:
		print(text, end = '')

print('..."')

In [ ]:
train_indexes, val_indexes = data.split_dataset(tokens)

print("Train indexes:", len(train_indexes))
print("Val indexes:", len(val_indexes))

In [ ]:
train_generator = BatchGenerator(tokens, train_indexes, val_frequency = VAL_FREQUENCY)
x, y = train_generator.__getitem__(0)

for i in range(len(x[0])):
	print(x[0][i], end = (max(len(str(x[0][i])), len(str(y[0][i]))) - len(str(x[0][i])) + 1) * " " + "| ")

print()

for i in range(len(y[0])):
	print(y[0][i], end = (max(len(str(x[0][i])), len(str(y[0][i]))) - len(str(y[0][i])) + 1) * " " + "| ")

print("\n")
print('"' + decode(tokenizer, x[0]) + '"')
print('"' + decode(tokenizer, y[0]) + '"')

In [ ]:
model = create_model(len(vocab))

model.compile(
	optimizer = AdamW(learning_rate = 0, weight_decay = WEIGHT_DECAY, beta_1 = BETA_1, beta_2 = BETA_2, global_clipnorm = CLIP_GRADIENTS),
	loss = 'sparse_categorical_crossentropy',
	metrics = ['accuracy']
)

model.summary()

In [ ]:
model.fit(
	train_generator,
	validation_data = BatchGenerator(tokens, val_indexes, ratio = 1.0 / VAL_FREQUENCY),
	batch_size = BATCH_SIZE,
	validation_batch_size = BATCH_SIZE,
	epochs = NUM_EPOCHS,
	shuffle = False,
	callbacks = [
		LRScheduler(len(train_indexes) // VAL_FREQUENCY),
		ModelCheckpoint(
			filepath = "model.h5",
			monitor = "val_loss",
			save_best_only = True,
			save_weights_only = True
		),
		EarlyStopping(
			monitor = "val_loss",
			patience = 20,
			restore_best_weights = True
		)
	]
)

In [ ]:
model.load_weights("model.h5")

In [ ]:
model.evaluate(BatchGenerator(tokens, val_indexes, ratio = 1.0 / VAL_FREQUENCY), batch_size = BATCH_SIZE)

In [ ]:
input = "<eod>"
output = predict(model, input, tokenizer, 256, 1, True)